In [38]:
# Installing dependencies
#!pip3 install --upgrade pip
#!pip3 install wget
#!pip3 install torch 
#!pip3 install torchvision 
#!pip3 install torchaudio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 9.4 MB/s eta 0:00:00a 0:00:01


In [66]:
#!pip3 install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cpu
!pip3 install --upgrade torch torchvision torchaudio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.2/53.2 MB 9.4 MB/s eta 0:00:0000:0100:01m
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0
    Uninstalling torch-1.11.0:
      Successfully uninstalled torch-1.11.0


In [1]:
# Importing dependencies
import torch
#from torch.backends import mps
print(torch.__version__)
import torchvision
import torch.nn as nn
import wget

from os.path import exists
from torch.nn import functional as F

1.13.1


In [2]:
torch.backends.mps.is_available()

True

In [3]:
# Hyperparameters & Constants
batch_size = 64
block_size = 256

# Number of iterations boosted to compensate for lower learning rates
max_iters = 5000
eval_interval = 500

# Self attention cannot tolerate very high learning rates well
learning_rate = 3e-4

#device = 'cuda' if torch.cuda.is_available() else 'cpu'

# For Mac M1 Max
device = 'mps' if torch.backends.mps.is_available() else 'cpu'

# Dropout takes a neural network and every forward and backwards pass across it shuts off subsets of nodes, neurons
# tokens, setting them to 0 and training without them, resulting in a training of an ensemble of networks, 
# 'merging' them at test time when all nodes are taken back online
# Used as a regularization technique when scaling up the model avoid overfitting
# Every forward and backward pass 20% intermediate calculations are disabled and dropped to 0
dropout = 0.2

eval_iters = 200
number_of_embedding_dimensions = 384

number_of_heads = 6
number_of_layers = 6

In [41]:
torch.cuda.is_available()

False

In [12]:
# Downloading the Tiny Shakespeare dataset
url = 'https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt'
file_name = wget.download(url)

100% [........................................................] 1115394 / 1115394

In [ ]:
# Saving the dataset to a text file
#with open('tiny_shakespeare.txt', 'wb') as file:
#    file.write(file_name)

In [158]:
if exists('input.txt') == True:
    print('Hi')

Hi


In [14]:
# Reading and inspecting the downloaded dataset
with open('input.txt', 'r', encoding = 'utf-8') as file:
    text = file.read()

In [15]:
print('Length of dataset via character count: ', len(text))

Length of dataset via character count:  1115394


In [16]:
# Observing the first 1000 characters of the text dataset
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [30]:
# Uncovering all of the unique characters which appear throughout the dataset
unique_characters = sorted(list(set(text)))
vocabulary_size = len(unique_characters)

print(f'Unique Characters: {"".join(unique_characters)} \n')
print(f'Vocabulary Size: {vocabulary_size}')

Unique Characters: 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz 

Vocabulary Size: 65


In [60]:
# Establishing an encoder, decoder tokenizer unique to this dataset by
# creating a mapping from characters to integers in the form of a dictionary

string_to_integer = {character:integer for integer, character in enumerate(unique_characters)}
integer_to_string = {integer:character for integer, character in enumerate(unique_characters)}

#print(type(string_to_integer))
#print(type(integer_to_string))

#print(f'{string_to_integer} \n')
#print(f'{integer_to_string} \n')

for dict1_key_value_pair, dict2_key_value_pair in zip(string_to_integer.items(), integer_to_string.items()):
    print(f'{dict1_key_value_pair} | {dict2_key_value_pair}')

('\n', 0) | (0, '\n')
(' ', 1) | (1, ' ')
('!', 2) | (2, '!')
('$', 3) | (3, '$')
('&', 4) | (4, '&')
("'", 5) | (5, "'")
(',', 6) | (6, ',')
('-', 7) | (7, '-')
('.', 8) | (8, '.')
('3', 9) | (9, '3')
(':', 10) | (10, ':')
(';', 11) | (11, ';')
('?', 12) | (12, '?')
('A', 13) | (13, 'A')
('B', 14) | (14, 'B')
('C', 15) | (15, 'C')
('D', 16) | (16, 'D')
('E', 17) | (17, 'E')
('F', 18) | (18, 'F')
('G', 19) | (19, 'G')
('H', 20) | (20, 'H')
('I', 21) | (21, 'I')
('J', 22) | (22, 'J')
('K', 23) | (23, 'K')
('L', 24) | (24, 'L')
('M', 25) | (25, 'M')
('N', 26) | (26, 'N')
('O', 27) | (27, 'O')
('P', 28) | (28, 'P')
('Q', 29) | (29, 'Q')
('R', 30) | (30, 'R')
('S', 31) | (31, 'S')
('T', 32) | (32, 'T')
('U', 33) | (33, 'U')
('V', 34) | (34, 'V')
('W', 35) | (35, 'W')
('X', 36) | (36, 'X')
('Y', 37) | (37, 'Y')
('Z', 38) | (38, 'Z')
('a', 39) | (39, 'a')
('b', 40) | (40, 'b')
('c', 41) | (41, 'c')
('d', 42) | (42, 'd')
('e', 43) | (43, 'e')
('f', 44) | (44, 'f')
('g', 45) | (45, 'g')
('h', 

In [111]:
# Encoder taking in a string and outputing a list of integers
encoder = lambda string: [string_to_integer[character] for character in string]

# Decoder taking in a list of integers and outputing a string
decoder1 = lambda list_of_integers: ''.join([integer_to_string[integer] for integer in list_of_integers])

def decoder2(encoded_string):
    decoded_string = ''
    
    for integer in encoded_string:
        #print(f'Integer to decode: {integer}')
        
        character = integer_to_string[integer]
        
        #print(f'Decoded integer: {character}')
        
        decoded_string += character
    
    return decoded_string

In [70]:
# There is a tradeoff between a tokenizer's overall total codebook size and the associated lengths of said
# codebook's sequences of integers

# Either very long sequences of integers with very small vocabularies
# or very short sequences of integers with very large vocabularies

# Typically most tokenizer dictionaries used are somewhere in the middle, making use of subwords

# As a character level tokenizer is being utilized, the sequences of integers lengths are very long, 
# but the size of the overall codebook is very small, as there are only 64 unique characters within 
# the Tiny Shakespeare dataset

test_string = 'Hello There!'
encoded_test_string = encoder(test_string)
decoded_test_string = decoder1(encoded_test_string)

print(f'Test String: {test_string}')
print(f'Encoded Test String: {encoded_test_string}')
print(f'Decoded Test String: {decoded_test_string}')

Test String: Hello There!
Encoded Test String: [20, 43, 50, 50, 53, 1, 32, 46, 43, 56, 43, 2]
Decoded Test String: Hello There!


In [72]:
# Encoding the entire text dataset whilst storing it within a PyTorch tensor
data = torch.tensor(encoder(text), dtype = torch.long)

In [73]:
# Some data about the tensor
print(f'Shape: {data.shape} | DType: {data.dtype} \n')

# Observing the first 1000 characters of the now encoded and tokenized dataset
print(data[:1000])

Shape: torch.Size([1115394]) | DType: torch.int64 

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 

In [74]:
# Splitting the data into the respective training and validation datasets
index = int(0.9 * len(data))

# Vast majority of the dataset for this example is used exclusively for training purposes
training_data = data[:index]

# Utilized to observe as to whether the model is overfitting, i.e. simply memorizing actual passages of
# Shakespeare rather than generating something which looks like Shakespeare
validation_data = data[index:]

In [75]:
# In real life scenarios and use cases, it is often prohibitively expensive or practically impossible to
# pass all of the relevant data into a transformer at once, and so it must be chunked into portions
# that said transformer will randomly select from throughout the available training data
# limited to a certain size or length defined by what is called block size or context length, for example
block_size = 8

In [77]:
# A chunk contains an equivalent of the block_size - 1 number of examples for the transformer to train upon

# For example, if a block contains the phrase 'Hello, my name is John.', a transformer using words as its tokens
# would see the example that the word 'Hello' is followed by the word 'my', and that the subphrase 'Hello my' is
# then followed by the word 'is', and so on and so forth

# Think of this as a dimension of time, what comes before and proceeds after something, giving context

# With this dataset, it is the individual characters which follow or proceed one another that serve as the basis
# for this process of learning by the transformer
training_data[:block_size + 1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [80]:
# For a deeper understanding, see below:
x = training_data[:block_size]
y = training_data[1:block_size + 1] # Index offset of 1 required to properly link a context to its target

for z in range(block_size):
    
    # The context is always all of the characters, words, etc. up to z
    context = x[:z + 1]
    
    # The target is always z
    target = y[z]
    
    print(f'When the input is: {context}\nThe target is: {target}\n')
    
# Transformers are trained in this fashion not just from an efficiency standpoint, as all of the data within
# the given chunk is already available at the given moment, but also as it gets the transformer used to seeing
# examples of various length and context from a length of 1 all the way up to the maximum given block size

# Once a transformer's predictions reach blocksize, the prediction is truncated as said transformer will never
# have been trained based upon a context greater than its given blocksize, inputs greater than the alloted
# blocksize will also be truncated or chunked appropriately to fit this restriction

When the input is: tensor([18])
The target is: 47

When the input is: tensor([18, 47])
The target is: 56

When the input is: tensor([18, 47, 56])
The target is: 57

When the input is: tensor([18, 47, 56, 57])
The target is: 58

When the input is: tensor([18, 47, 56, 57, 58])
The target is: 1

When the input is: tensor([18, 47, 56, 57, 58,  1])
The target is: 15

When the input is: tensor([18, 47, 56, 57, 58,  1, 15])
The target is: 47

When the input is: tensor([18, 47, 56, 57, 58,  1, 15, 47])
The target is: 58



In [94]:
# Batch dimension is also a consideration for transformers

# Many batches of multiple chunks of text will be fed all at once into a transformer, all wrapped up within a 
# single tensor for the purposes of efficiency, as GPUs are designed to deal with these sorts of inputs 
# processing them in parallel during the transformer's training

# These chunks are processed completly independently and never communicate with one another during this process

# Seed set for reproducibility of results due to the random sampling of chunks from throughout the training data
torch.manual_seed(1337)

# The number of independent sequences which will be processing in parallel
#batch_size = 4

# The maximum content length for predictions
#block_size = 8

In [124]:
def get_batch(split):
    
    # Generates a small batch of data of 'x' inputs and 'y' targets
    
    # If the split is a training split, it will look through the training data, otherwise it will look into
    # the validation data
    data = training_data if split == 'training' else validation_data
    
    # Generates a tensor of random offsets within the chosen dataset based off of the desired batch size
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    
    # The first block size characters starting at an index of 'i'
    # Each of these 'x' chunks are retrieved for every randomized index starting point of 'i' taken from 'ix'
    # These chunks of 1 dimensional tensors are then stacked as rows within a tensor table of dimensions
    # batch size * block size
    x = torch.stack([data[i:i + block_size] for i in ix])
    
    # Same as x but offset by 1, the target from the context given by x
    y = torch.stack([data[i + 1:i + block_size + 1] for i in ix])
    
    # Ensuring that if a GPU is being ultilized or is available, that the data is moved over to that
    # device before progressing on
    x, y = x.to(device), y.to(device)
    
    return x, y

In [123]:
# This example spells out how there will be 32 independent examples of input 'x' which will be fed into a
# transformer, which will reference the input 'y' as a basis for its future predictions
xb, yb = get_batch('training')

print('Inputs: ')
print(f'Batch Shape: {xb.shape}')
print(xb)
print('\n')

print('Targets: ')
print(f'Batch Shape: {yb.shape}')
print(yb)
print('\n')

print('----------')

Inputs: 
Batch Shape: torch.Size([32, 8])
tensor([[56, 43, 58, 53, 50, 42,  1, 58],
        [39, 49, 43,  1, 43, 45, 45, 57],
        [ 1, 52, 53,  1, 40, 43, 39, 57],
        [51,  1, 46, 39, 60, 43,  1, 41],
        [ 1, 46, 39, 60, 43,  1, 40, 56],
        [ 1, 53, 44, 44,  8,  0, 21, 58],
        [ 6,  1, 58, 46, 43,  1, 46, 43],
        [50, 43, 57, 57,  1, 54, 53, 61],
        [58, 46, 43,  1, 40, 50, 39, 51],
        [26, 43, 39, 54, 53, 50, 47, 58],
        [59, 57, 58,  1, 47, 58,  1, 40],
        [10,  0, 21,  1, 42, 39, 56, 43],
        [ 6,  1, 47, 44,  1, 58, 46, 63],
        [56, 11,  1, 46, 43, 56, 43,  1],
        [10,  0, 15, 39, 52,  1, 57, 47],
        [42,  1, 47, 58,  1, 52, 53, 58],
        [39, 58, 46,  1, 52, 53, 58,  1],
        [46, 47, 57,  1, 40, 43, 50, 50],
        [43, 39, 56,  1, 51, 43,  6,  1],
        [42, 10,  1, 40, 59, 58,  1, 46],
        [53, 59,  1, 44, 50, 39, 58, 58],
        [52,  6,  0, 35, 43, 56, 43,  1],
        [59, 58,  1, 51, 43, 43, 4

In [97]:
# Spelling it out
for b in range(batch_size):
    for t in range(block_size):
        
        context = xb[b, :t + 1]
        target = yb[b, t]
        
        print(f'When the input is: {context}\nThe target is: {target}\n')

When the input is: tensor([24])
The target is: 43

When the input is: tensor([24, 43])
The target is: 58

When the input is: tensor([24, 43, 58])
The target is: 5

When the input is: tensor([24, 43, 58,  5])
The target is: 57

When the input is: tensor([24, 43, 58,  5, 57])
The target is: 1

When the input is: tensor([24, 43, 58,  5, 57,  1])
The target is: 46

When the input is: tensor([24, 43, 58,  5, 57,  1, 46])
The target is: 43

When the input is: tensor([24, 43, 58,  5, 57,  1, 46, 43])
The target is: 39

When the input is: tensor([44])
The target is: 53

When the input is: tensor([44, 53])
The target is: 56

When the input is: tensor([44, 53, 56])
The target is: 1

When the input is: tensor([44, 53, 56,  1])
The target is: 58

When the input is: tensor([44, 53, 56,  1, 58])
The target is: 46

When the input is: tensor([44, 53, 56,  1, 58, 46])
The target is: 39

When the input is: tensor([44, 53, 56,  1, 58, 46, 39])
The target is: 58

When the input is: tensor([44, 53, 56,  1,

In [98]:
# Example input into a transformer
print(xb)

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


In [160]:
# Bigram Language Model
# One of the simplest neural network language models

# Custom subclass of the 'nn' module
class BigramLanguageModel(nn.Module):
    
    # Constructor
    # Technically no need to pass in the 'vocabulary_size' variable into the constructor at it already
    # exists above as a global variable, kept in as a personal preference
    def __init__(self, vocabulary_size):
        super().__init__()
        
        # Creating a token embedding table
        # There is a very thin wrapper around a tensor of shape vocabulary size * vocabulary size
        # Each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocabulary_size, vocabulary_size)
        
    def forward(self, idx, targets = None):
        
        # When an index is passed in, every single integer from the input will refer to the embedding
        # table and will pluck out a row of said table coresponding to its index
        # An example would be an object tokenized as the integer number 24 would select the row with index
        # of 24 from the embedding table
        
        # 'idx' and 'targets' are both (Batch, Time) tensor of integers
        
        # (Batch, Time, Channel) Tensor will be arranged by PyTorch
        # (4, 8, 65 the vocabulary_size)
        # Logits are the scores for the next characters in the sequence
        # A prediction for what comes next is based off of the single individual identity of a token
        logits = self.token_embedding_table(idx)
        
        # Loss is the crossentropy of the predictions upon the targets
        # Measures the quality of the logits with respect to the targets
        # We have the actual identity of the next character, how well is the next character being predicted 
        # based upon the logits
        if targets is None:
            loss = None
        
        else:
            # For loss calculations to run, PyTorch requires an input of form (Batch, Channel, Time) 
            # for multidimensional inputs
            B, T, C = logits.shape

            # Reducing a 3D array to a 2D array, stretching out the 2D 'xb' input into a 1D column, 
            # preserving the Channel as the second 'column' to better conform to Pytorch's requirements
            logits = logits.view(B * T, C)

            # Targets must also be reshaped as a result
            targets = targets.view(B * T) # Explicit
            #targets = targets.view(-1) # Otherwise PyTorch will attempt to guess the best shape out the output

            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    # The purpose of this function is to continue the generation in all of the batch dimensions in the time 
    # dimension of (B, T) with relation to 'idx', so that you end up with (B, T + 1), (B, T + 2), etc.
    def generate(self, idx, max_new_tokens):
        
        #idx is (B, T), an array of indices in the current context
        for _ in range(max_new_tokens):
            
            # Retrieving the predictions from the current index
            logits, loss = self(idx)
            
            # Focusing solely upon the last time step
            # Removing the last element of the time dimension as those are the predictions for what comes next
            logits = logits[:, -1, :] # (B, T) becomes (B, C)
            
            # Applying softmax in order to obtain probabilities via conversion of the logits
            probs = F.softmax(logits, dim = -1) # (B, C)
            
            # Sampling from the distribution of probabilities, retrieving only 1 sample from PyTorch
            # For each of the batch dimensions, there will now only be a single prediction for what comes next
            idx_next = torch.multinomial(probs, num_samples = 1) # (B, 1)
            
            # Appending sampled index to the running sequence
            
            # Whatever is predicted from 'idx_next' is concatenated on top of the previous 'idx' along
            # the first dimension, the Time dimension
            
            # Takeing the integers from the above line's sampling process, concatenating them on top of the
            # running steam of integers
            idx = torch.cat((idx, idx_next), dim = 1) # (B, T + 1)
            
        return idx

In [31]:
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        
        # Linear projections being applied to all tokens and nodes
        self.key = nn.Linear(number_of_embedding_dimensions, head_size, bias = False)
        self.query = nn.Linear(number_of_embedding_dimensions, head_size, bias = False)
        self.value = nn.Linear(number_of_embedding_dimensions, head_size, bias = False)
        
        # 'tril' is not a parameter of the model, it is a buffer via PyTorch naming conventions
        # Lower triangular matrix
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        batch, time, channels = x.shape
        
        # Calculating token keys
        keys = self.key(x) # Shape of (batch, time, channels)
        
        # Calculating token queries
        queries = self.query(x) # Shape of (batch, time, channels)
        
        # Computing attention scores, i.e. affinities
        # (batch, time, channels) @ (batch, channels, time) -> (batch, time, time)
        # Calculating attention scores within the weights, normalizing them via scaled attention
        weights = queries @ keys.transpose(-2, -1) * channels ** -0.5
        
        # Ensuring future tokens cannot communicate with past tokens
        weights = weights.masked_fill(self.tril[:time, :time] == 0, float('-inf')) # Shape of (batch, time, time)
        weights = F.softmax(weights, dim = -1) # Shape of (batch, time, time)
        
        # Can potentially dropout, preventing random communication from nodes to avoid overfitting
        weights = self.dropout(weights)
        
        # Performing the weighted aggregation of the values
        values = self.value(x) # Shape of (batch, time, channels)
        
        # (batch, time, time) @ (batch, time, channels) -> (batch, time, channels)
        output = weights @ values
        
        return output

In [29]:
# Multi Head Attention
# Multiple heads are working in parallel in order to concatenate the relevant results
class MultiHeadAttention(nn.Module):
    def __init__(self, number_of_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(number_of_heads)])
        self.projection = nn.Linear(number_of_embedding_dimensions, number_of_embedding_dimensions)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        # Concatenating results over the channel dimension
        
        # Output of the self attention
        output = torch.cat([head(x) for head in self.heads], dim = -1)
        
        # Application of the projection onto the self attention output
        # Projection is a linear transformation of the outcome of the previous layer
        # Can potentially dropout at the multi attention head stage
        output = self.dropout(self.projection(output))
        
        # Projection will be fed back into the residual pathway
        return output

In [30]:
# Primary Revised Bigram Language Model
class BigramLanguageModel2(nn.Module):
    def __init__(self, vocabulary_size):
        super().__init__()
        
        # Instead of going diectly towards the embedding phase, a level of interaction is desired
        # between the logits
        self.token_embedding_table = nn.Embedding(vocabulary_size, number_of_embedding_dimensions)
        
        # Encoding indeces based upon the position of their tokens on top of the tokens' identities
        # Each position from 0 to block size - 1 will also get their own embedding vector
        self.postion_embedding_table == nn.Embedding(block_size, number_of_embedding_dimensions)
        
        # Declaration of head size
        self.self_attention_head = Head(number_of_embedding_dimensions)
        
        # Logits are no longer aquired directly, instead first token embeddings are retrieved
        # In order to go from the token embeddings to the logits, a linear layer is required
        self.language_modeling_head = nn.Linear(number_of_embedding_dimensions, vocabulary_size)
        
    def forward(self, index, targets = None):
        batch, time = index.shape
        
        # Indeces are encoded based upon the identity of the tokens
        
        # 'index' and 'targets' are both (batch, time) shaped tensors of integers
        # Logits are no longer aquired directly, instead first token embeddings are retrieved
        
        # Right aligned, a new dimension of 1 gets added, and then broadcasted across batch
        token_embeddings = self.token_embedding_table(index) # Shape (batch, time, channels_1)
        position_embeddings = self.position_embedding_table(torch.arange(time, device = device)) # (time, channels)
        
        # Holds the token identities as well as the positions at which said tokens occur
        token_id_position_embeddings = token_embeddings + position_embeddings # (batch, time, channels)
        
        # Applying one head of self attention to the composite token embeddings
        # Shape of (batch, time, channels)
        token_id_position_embeddings = self.self_attention_head(token_id_position_embeddings)
        
        # Care must be taken, as the channels dimensions are no longer equal, as 
        # channels_1 = number_of_embedding_dimensions and channels_2 = vocabulary size
        logits = self.language_modeling_head(token_id_position_embeddings) # Shape (batch, time, channels_2)

        if targets is None:
            loss = None
        
        else:
            batch, time, channels = logits.shape
            logits = logits.view(batch * time, channels)
            targets = targets.view(batch * time)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            
            # As positional embedding is now in usage, never more than the set block size is allowed to
            # come in at any one time, as since if an index is larger than the block size, an out of scope
            # error will arise due to the position embedding table only going up to said block size
            index_condition = index[:, -block_size:]
            
            logits, loss = self(index_condition)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim = -1)
            index_next = torch.multinomial(probs, num_samples = 1)
            index = torch.cat((index, index_next), dim = 1)
            
        return index

In [27]:
# Feed Forward
# Essentially just a multi layer perceptron
# Adding computation into the neural network at a per node level
class FeedForward(nn.Module):
    def __init__(self, number_of_embedding_dimensions):
        super().__init__()
        
        # Tokens original had time to look at one another but not enough time to truly 'think on'
        # and contextualize said information, which is why these layers are added
        # Occurs on a per token level, evey token and node doing this independently
        self.net = nn.Sequential(
            # The inner layer of the feed forward network should be 4 times as large as the output in terms
            # of channel sizes
            nn.Linear(number_of_embedding_dimensions, 4 * number_of_embedding_dimensions),
            nn.ReLU(),
            
            # Projection layer going back into the residual pathway, shrinked back down to the proper size
            nn.Linear(4 * number_of_embedding_dimensions, number_of_embedding_dimensions),
            
            # Occurs right before the merging back into the residual pathway
            nn.Dropout(dropout),
        )
        
    def forward(self, x):
        return self.net(x)

In [21]:
# Layer Normalization
# Normalizes the rows of every example individually
# It used to be that the addition and normalization portions of transformers were carried out after
# transformation, but typically now the normalization step is taken before said transformations
class BatchNorm:
    def __init__(self, dim, eps = 1e-5, momentum = 0.1):
        self.eps = eps
        self.gamma = torch.ones(dim)
        self.beta = torch.zeros(dim)
    
    def __call__(self, x):
        # Calculating the forward pass
        
         # Batch mean
        x_mean = x.mean(1, keepdim = True)
        
         # Batch variance
        x_variance = x.var(1, keepdim = True)
        
         # Normalizing to unit variance
        x_hat = (x - x_mean) / torch.sqrt(x_variance + self.eps)
        
        self.out = self.gamma * x_hat + self.beta
        
        return self.out
    
    def parameters(self):
        return [self.gamma, self.beta]

In [16]:
# Secondary Revised Bigram Language Model
# Use of multi head attention greatly reduces the final trained model's overall loss
class BigramLanguageModel3(nn.Module):
    def __init__(self, vocabulary_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocabulary_size, number_of_embedding_dimensions)
        self.postion_embedding_table == nn.Embedding(block_size, number_of_embedding_dimensions)
        
        # 4 Heads of 8 dimensional self attention that when concatenated results in a total of 32, the
        # number of embedded dimensions, similar to group convolution
        # 4 communications channels acting in parallel
        self.self_attention_heads = MultiHeadAttention(4, number_of_embedding_dimensions // 4)
        
        # Enables the tokens to better contextualize and learn from the other tokens they are observing
        # during the learning process
        self.feed_forward = FeedForward(number_of_embedding_dimensions)
        
        self.language_modeling_head = nn.Linear(number_of_embedding_dimensions, vocabulary_size)
        
    def forward(self, index, targets = None):
        batch, time = index.shape
        token_embeddings = self.token_embedding_table(index)
        position_embeddings = self.position_embedding_table(torch.arange(time, device = device))
        token_id_position_embeddings = token_embeddings + position_embeddings
        token_id_position_embeddings = self.self_attention_heads(token_id_position_embeddings)
        
        # Self attention is the communication gathering data, the feed forward is thinking on said data
        # for each token, further reduces loss during model training
        # Shape of (batch, time, channels)
        token_id_position_embeddings = self.feed_forward(token_id_position_embeddings)
        
        logits = self.language_modeling_head(token_id_position_embeddings)

        if targets is None:
            loss = None
        
        else:
            batch, time, channels = logits.shape
            logits = logits.view(batch * time, channels)
            targets = targets.view(batch * time)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            index_condition = index[:, -block_size:]
            logits, loss = self(index_condition)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim = -1)
            index_next = torch.multinomial(probs, num_samples = 1)
            index = torch.cat((index, index_next), dim = 1)
            
        return index

In [25]:
# Transformer Block
# Interspersing communication with computation
# Same act a transformer carries out using blocks to communicate and compute
class Block(nn.Module):
    
    # Communication followed by computation
    def __init__(self, number_of_embedding_dimensions, number_of_heads):
        super().__init__()
        
        # Should be a head size 8 so that 4 blocks result in an overall dimension of 32
        head_size = number_of_embedding_dimensions // number_of_heads
        
        # Communication
        self.self_attention = MultiHeadAttention(number_of_heads, head_size)
        
        # Computation
        self.feed_forward = FeedForward(number_of_embedding_dimensions)
        
        # Mean and variance are taken over the number of embedding dimensions, in our case 32 numbers
        # Acting as a per token transformation, normalizing the features and rendering them 
        # unit mean and unit gautian at initialization, though this may change as the gamma and beta
        # variables within during training might be optimized to change this
        self.layer_normalization1 = nn.LayerNorm(number_of_embedding_dimensions)
        self.layer_normalization2 = nn.LayerNorm(number_of_embedding_dimensions)
    

    # Skip & Residual Connections

    # Transform data with a skip connection adding data from previous features
    # Residual pathway is forked off, calculations and transformations carried out on fork, then recombined
    # with the residual pathway via addition

    # Addition distributes gradients equally to oth branches in terms of backpropagation

    # Gradient superhighway from the supervision to the input, unimpeded, blocks kick in over time, having minimal
    # impact at the start but becoming more important as training progresses, aiding in optimization by reducing
    # the load
    def forward(self, x):
        # Forking off the residual path to carry out some communication then coming back
        x = x + self.self_attention(self.layer_normalization1(x))
        
        # Forking off the residual path to carry out some computation then coming back
        x = x + self.feed_forward(self.layer_normalization2(x))
        
        return x

In [34]:
# Tertiary Revised Bigram Language Model
# Almost reaching the point of beginning to see overfitting due to the increasing size of the neural network
class BigramLanguageModel4(nn.Module):
    
    # Decoder only transformer
    def __init__(self, vocabulary_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocabulary_size, number_of_embedding_dimensions)
        self.postion_embedding_table == nn.Embedding(block_size, number_of_embedding_dimensions)
        
        # At this point in time, the neural network is becoming relatively deep, resulting in 
        # optimization issues and a reduced level of improvement compared to past modifications
        #self.blocks = nn.Sequential(
        #    Block(number_of_embedding_dimensions, number_of_heads = 4), 
        #    Block(number_of_embedding_dimensions, number_of_heads = 4), 
        #    Block(number_of_embedding_dimensions, number_of_heads = 4), 
            
            # Normalization layer right at the end of the transformer and right before the decoding layer
        #    nn.LayerNorm(number_of_embedding_dimensions), 
        #)
        
        # For scaled version
        self.blocks = nn.Sequential(*[Block(
                                      number_of_embedding_dimensions, 
                                      number_of_heads = number_of_heads) 
                                      for _ in range(number_of_layers)])
        
        # Final layer normalization
        self.layer_normalization = nn.LayerNorm(number_of_embedding_dimensions)
        
        self.language_modeling_head = nn.Linear(number_of_embedding_dimensions, vocabulary_size)
        
    def forward(self, index, targets = None):
        batch, time = index.shape
        token_embeddings = self.token_embedding_table(index)
        position_embeddings = self.position_embedding_table(torch.arange(time, device = device))
        token_id_position_embeddings = token_embeddings + position_embeddings
        
        # Shape of (batch, time, channels)
        token_id_position_embeddings = self.blocks(token_id_position_embeddings) 
        
        token_id_position_embeddings = self.layer_normalization(token_id_position_embeddings)
        
        logits = self.language_modeling_head(token_id_position_embeddings)

        if targets is None:
            loss = None
        
        else:
            batch, time, channels = logits.shape
            logits = logits.view(batch * time, channels)
            targets = targets.view(batch * time)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            index_condition = index[:, -block_size:]
            logits, loss = self(index_condition)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim = -1)
            index_next = torch.multinomial(probs, num_samples = 1)
            index = torch.cat((index, index_next), dim = 1)
            
        return index

In [125]:
model = BigramLanguageModel(vocabulary_size)

# Ensure that the model is also moved over to the appropirate device
model = model.to(device)

logits, loss = model(xb, yb)

# This tensor contains the scores for every possible character which may come next
print(logits.shape)

# Loss is slightly higher than desired, meaning the guessing are wrong, but this is to be exepected
# as the initial predictions are not yet diffuse and have a little bit of entropy
# Loss of ~4.53 as opposed to ln(1 / 65) =  ~4.17
print(loss)

torch.Size([256, 65])
tensor(4.7652, grad_fn=<NllLossBackward0>)


In [112]:
# (Batch = 1, Time = 1), 1 * 1 tensor holding a 0, used to kick off the generation and acting as the 
# first character of a sequence
idx = torch.zeros((1, 1), dtype = torch.long)

# As generate() works on the level of batches, the lone 0 tensor must be removed by indexing into the 0 throw
# This ultimately generates a 1 dimensional row of indices that will then be converted into a simple Python list
# This list from a tensor can then be decoded to covert the integers to characters
print(decoder1(model.generate(idx, max_new_tokens = 100)[0].tolist()))

# Result is garbage as the model is not yet trained and so outputs something completely random
# Example Output:
#qvE33zoVPKeqAIldEzdXCJCoh'YE?JU'dP&uVu;ep
#;SwN&ubd-iND,.F!
#UlMH;fSUyru.TfSLiyek,pA D!pF$AfF'AVNKlepA

# As is, the model only looks at the immediately preceding character to make a prediction, and all other
# characters in the context's history are ignored from the perspective of a bigram model, this will change
# further on to avoid by so inefficient for no reason


qvE33zoVPKeqAIldEzdXCJCoh'YE?JU'dP&uVu;ep
;SwN&ubd-iND,.F!
UlMH;fSUyru.TfSLiyek,pA D!pF$AfF'AVNKlepA


In [114]:
# Creating a PyTorch optimizer
# Learning rate is extremly low compared to most models due to how small the actual amount of data
# is made available, allowing for such values
optimizer = torch.optim.AdamW(model.parameters(), lr = 1e-3)

In [116]:
# Basic training loop
batch_size = 32

for steps in range(100000):
    
    # Sampling a batch of data
    xb, yb = get_batch('training')
    
    # Evaluation the loss
    logits, loss = model(xb, yb)
    
    # Zeroing out all gradients from the previous step
    optimizer.zero_grad(set_to_none = True)
    
    # Retrieving the gradients for all of the current step's parameters
    loss.backward()
    
    # Utilizing the newly retrieved gradients to update said parameters
    optimizer.step()
    
print(loss.item())

2.4083828926086426


In [117]:
# Results are hardly perfect, but seem to be making progress
print(decoder1(model.generate(idx, max_new_tokens = 100)[0].tolist()))

# Example Output:
#I touarrtan
#SCave t, wize abe inrayetro e d or's knchen mmanthearomefonther y din:
#We have m RIfathe


I touarrtan
SCave t, wize abe inrayetro e d or's knchen mmanthearomefonther y din:
We have m RIfathe


In [120]:
# PyTorch does not find a GPU on Mac M1 Max, but already have something installed
print(torch.cuda.is_available())

False


In [127]:
# Averages out the loss over multiple batches
# Avoids the noisy version of simply printing out of the loss directly from within the training loop
# for the generator

# Context manager indicating to PyTorch that everything occuring within this function will not be called
# backward on for backpropagation,  making it far more efficient memory wise be not having to store all 
# those intermediate variables
@torch.no_grad()
def estimate_loss():
    out = {}
    
    # Sets the model to the evaluation phase
    # Important to take this into account, as neural network model layers have varying behavior between
    # training and evaluation phases
    model.eval()
    
    for split in ['training', 'validation']:
        losses = torch.zeros(eval_iters)
        
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        
        out[split] = losses.mean()
    
    # Sets the model to the training phase
    model.train()
    
    return out

In [130]:
# Proper training loop
for iter in range(max_iters):
    
    # Evaluating losses on both the training and validation data sets every so often
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"Step {iter}: Training Loss = {losses['training']:.4f} | Validation Loss = {losses['validation']:.4f}")
    
    # Sampling a batch of data
    xb, yb = get_batch('training')
    
    # Evaluating the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none = True)
    loss.backward()
    optimizer.step()

Step 0: Training Loss = 4.6858 | Validation Loss = 4.7224
Step 300: Training Loss = 4.6879 | Validation Loss = 4.7171
Step 600: Training Loss = 4.6893 | Validation Loss = 4.7289
Step 900: Training Loss = 4.6978 | Validation Loss = 4.7212
Step 1200: Training Loss = 4.6895 | Validation Loss = 4.7189
Step 1500: Training Loss = 4.6937 | Validation Loss = 4.7136
Step 1800: Training Loss = 4.6944 | Validation Loss = 4.7216
Step 2100: Training Loss = 4.6979 | Validation Loss = 4.7175
Step 2400: Training Loss = 4.6936 | Validation Loss = 4.7160
Step 2700: Training Loss = 4.6854 | Validation Loss = 4.7240


In [131]:
# Generating from the model
context = torch.zeros((1, 1), dtype = torch.long, device = device)
print(decoder1(model.generate(context, max_new_tokens = 500)[0].tolist()))


mVS&o!qd-CvZA.sE
akcU
lWX.kv-UC:cugYeaNuc,tnyeHKf;W,s bHH!qdvKr&e;WtfL&qzENuLBNLizNx HsA,mg:clUkz$XuyBXOrNSxJ
G!bNn!qSqh!quXSxGuLu!qsbcS
usJTYFFxrggRWMKfVvWQnZ?Lj,S:&fW&r?EDXG-iP.rhFKlGypK'l$yMX&,fGggqu
;e,XGoHkPXO-i.rXVX.LoJm:c,?:VrNqItyZJSSxdMBnnKrUMzZDgg YVp3AVS:cpZTUJSwsZKjYF-ulh:TEsZVtq,.auZAjwn:EDBs'hqBAvIL!Xu3gLCXOehoo:cVI vvH.krPKxTQK$I-?P.ajkfLl'BNhwnPvtZmHP'OAV.sZojm$EA.ZqvI'V&3hul!qfVGowuVmC,Vv,SxZybrBe:Th--!XwoRrbOrd,;VXAe:YAG
GH!LRoLVTz quZZFxp?rGusTEm3YTzYQgggBNPKmsEmHKFRpvlQJxz,mX


In [132]:
# Mathematical trick in self attention
# Desire for the tokens from the time dimension to speak to all those tokens which came before them, but not
# to those which came after

# Calculating the average of all the vectors from the previous and current token is one method to enable
# interaction, but it is extremely weak, as a lot of information is lost such as positioning

# Toy Example
torch.manual_seed(1337)
B, T, C = 4, 8 ,2 # Batch, Time, CHannels
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

In [133]:
# Initialized at zero
x_bag_of_words = torch.zeros((B, T, C))

# Iterating over every batch dimension independently
# For loops are inneficient, better to carry out this process via matrix multiplication
for b in range(B):
    
    # Iterating over time
    for t in range(T):
        
        # Previous tokens include everything within the batch dimension up to the time
        # The chunk of tokens previous to the current token for the given sequence
        xprev = x[b, :t + 1] # Shape (t elements in past, C)
        
        # Averaging out the time dimension, resulting in a 1D (C) vector
        x_bag_of_words[b, t] = torch.mean(xprev, 0)
        
# Every new row is now the average of the previous rows

In [134]:
x[0]

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])

In [136]:
x_bag_of_words[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [140]:
# Can quickly accomplish this via matrix multiplication
a1 = torch.ones(3, 3)
a2 = torch.tril(a1)

# Generating a matrix with multiplication coefficients that when used in matrix multiplication return
# a final matrix of progressing incremental averages
a2 = a2 / torch.sum(a2, 1, keepdim = True)

b = torch.randint(0, 10, (3, 2)).float()
c = a2 @ b

print(a1)
print(a2)
print(b)
print(c)

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
tensor([[8., 9.],
        [2., 7.],
        [3., 9.]])
tensor([[8.0000, 9.0000],
        [5.0000, 8.0000],
        [4.3333, 8.3333]])


In [143]:
weights = torch.tril(torch.ones(T, T))
weights = weights / weights.sum(1, keepdim = True)

# Batch matrix multiplication will be done in parallel and individually, weighted aggregation
# Weight matrix @ data matrix
# (B, T, T) @ (B, T, C) = (B, T, C)
x_bag_of_words = weights @ x

In [142]:
weights

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [144]:
x_bag_of_words

tensor([[[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]],

        [[ 1.3488, -0.1396],
         [ 0.8173,  0.4127],
         [-0.1342,  0.4395],
         [ 0.2711,  0.4774],
         [ 0.2421,  0.0694],
         [ 0.0084,  0.0020],
         [ 0.0712, -0.1128],
         [ 0.2527,  0.2149]],

        [[-0.6631, -0.2513],
         [ 0.1735, -0.0649],
         [ 0.1685,  0.3348],
         [-0.1621,  0.1765],
         [-0.2312, -0.0436],
         [-0.1015, -0.2855],
         [-0.2593, -0.1630],
         [-0.3015, -0.2293]],

        [[ 1.6455, -0.8030],
         [ 1.4985, -0.5395],
         [ 0.4954,  0.3420],
         [ 1.0623, -0.1802],
         [ 1.1401, -0.4462],
         [ 1.0870, -0.4071],
         [ 1.0430, -0.1299],
         [ 1.1138, -0.1641]]])

In [146]:
# Introducing softmax

# Starting with a tensor of shape (T * T) of ones
tril = torch.ones(T, T)

# Applying the tril function to the tensor to obtain a diagonal fill of ones
tril = torch.tril(tril)
tril

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [148]:
# Starting with a tensor of zeros of shape (T * T) for the weights tensor
# How many tokens are desired for aggregation and averaging so as to transfer information
# In usage, some tokens will find others more or less interesting, affinity is affected and will
# have an impact on the final output
weights = torch.zeros((T, T))
weights

tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]])

In [150]:
# For all elements within the tril tensor = 0, fill in the appropriate 
# elements within the weights tensor with '-inf'
# Tokens on '-inf' will not be aggregated, tokens from the past cannot communicate with those of the future
weights = weights.masked_fill(tril == 0, float('-inf'))
weights

tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0., 0., 0.]])

In [151]:
# Applies a softmax aggregation across every single row of the weights tensor'
# Exponentiates every element in a row, so 0 becomes 1 and -inf becomes 0, then these are averaged
weights = F.softmax(weights, dim = -1)
weights

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [153]:
xbow3 = weights @ x
torch.allclose(x_bag_of_words, xbow3)

True

In [162]:
# Self Attention
torch.manual_seed(1337)

# A 4 by 8 arrangement of tokens, with each token having information be 32 dimensional
batch, time, channels = 4, 8, 32
x = torch.randn(batch, time, channels)

# Carries out simple average of all past tokens and the current token
tril = torch.tril(torch.ones(time, time))

# Affinities are being initialized at 0, losing out on certain tokens being more or less interested in others
# An example would be a consonate not knowing if another consonate or vowel came just before it to be able
# to later on make a better prediction, which is lost in this method
weights = torch.zeros((time, time))
weights = weights.masked_fill(tril == 0, float('-inf'))
weights = F.softmax(weights, dim = -1)
output = weights @ x

output.shape

torch.Size([4, 8, 32])

In [166]:
# Attention is a method of communication, with vectors of information lying around in space, tokens / nodes
# aggregating the information being pointed towards them by other tokens
# Compared to CNNs, there is no concept of space, orientation, positioning, etc., so in order to have said
# tokens take this into consideration, the information must in some way be included in the vectors of information

# Example of character based word creation for the word 'home': Tokens pointing with context to space and position
# h -> o
# h and o -> m
# h and o and m -> e

# Tokens want information to flow to them that they find most interesting or useful
# Gather information from the past in a data dependent way

# Every token will emit 2 vectors, a query and a key
# Query vector is what is the token looking for
# Key vector is what the token contains

# The affinities formed between tokens now form from the dot product 
# of a query upon all the keys of the other tokens, become weights

# If a key and query align well, than more will be learned between the 
# 2 involved tokens than any others in the sequence

torch.manual_seed(1337)
batch, time, channels = 4, 8, 32
x = torch.randn(batch, time, channels)

# Single head performing self attention
head_size = 16 # Hyperparameter

# Bias set to false to avoid matrix mutliplication with fixed weights
key = nn.Linear(channels, head_size, bias = False)
query = nn.Linear(channels, head_size, bias = False)
value = nn.Linear(channels, head_size, bias = False)

# When linear key and query torches are forwarded atop of the 'x' tensor of tokens, all tokens in all positions
# of the batch by time arrangement will independently, in parallel, and without communicating create their 
# own unique key and query
k = key(x) # Shape of (batch, time, head_size)
q = query(x) # Shape of (batch, time, head_size)

# All queries must now dot product with all the keys to produce affinity
# Avoid the batch dimension
# For every row of a batch, there will be a time squared matrix providing the affinities as weights
weights = q @ k.transpose(-2, -1) # (batch, time, head_size) @ (batch, head_size, time) = (batch, time, time)

tril = torch.tril(torch.ones(time, time))


# Encoder Block
# Masking used to hide tokens in the 'future' from those in the past to avoid affinity which shouldn't be possible

# In cases of sentiment analysis, for example, you might want future words to talk to past words so as to be
# able to understand concepts such as sarcasm or what not, as words in the future of a piece of text may invalidate
# or spin the meaning of words in the past despite what their typical usage might imply, the 'BUT'
weights = weights.masked_fill(tril == 0, float('-inf'))

# Normalizing values to inform the current token how much information to aggregate from those in the past
weights = F.softmax(weights, dim = -1)

# 'v' is the vector being aggregated rather than the raw 'x'
# 'x' is information private to only an individual token, the key being what it has, the query being what
# it is interested in, and the value being what another token might want to know if they are interest in it
v = value(x)
output = weights @ v
#output = weights @ x

output.shape

torch.Size([4, 8, 16])

In [165]:
# Weights are no longer constants, they are unique as each batch element have differing tokens
# Now data dependent
weights

tensor([[[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
         [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
         [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
         [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],

        [[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1687, 0.8313, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2477, 0.0514, 0.7008, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.4410, 0.0957, 0.3747, 0.0887, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0069, 0.0456, 0.0300, 0.7748, 0.1427, 0.0000, 0.0000, 0.0000],
         [0.0660, 0.089

In [ ]:
# Self attention is when the source of the token's keys, queries and values all originate from the same source

# Cross attention is where there is a seperate source of tokens off to the side where information is pulled from
# outside of the current data and only the queries are local, the keys and values coming from an external source
# and sometimes encoder blocks based on a preconditioned context

In [42]:
# To properly carry out the theoretical to the practical, technically a scaled attention should be applied, 
# however since weights are then fed into softmax in this demonstration, the weights should be fairly diffuse
# going in, as if there are opposite extremes of very positive and negative values, than softmax will force
# a convergence towards 1 hot vectors, essentially ignoring all other tokens or nodes in terms of aggregating
# information

# Scaling used to control variance at initialization
batch, time, head_size = 4, 8, 16

k = torch.randn(batch, time, head_size)
q = torch.randn(batch, time, head_size)
weights = q @ k.transpose(-2, -1) * head_size ** -0.5

print(k.var())
print(q.var())
print(weights.var())

tensor(1.0001)
tensor(0.9428)
tensor(1.1663)


In [43]:
# Diffuse weights
print(torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim = -1))

# One hot weights, sharpened towards the originally highest valued weight via an example multiplication
print(torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]) * 8, dim = -1))

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])
tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])


In [44]:
# Layer Normalization
torch.manual_seed(1337)
module = BatchNorm(100)
x = torch.randn(32, 100)
x = module(x)

print(x.shape)

torch.Size([32, 100])


In [45]:
# Mean and standard deviation of one feature across all batch inputs
x[:, 0].mean(), x[:, 0].std()

(tensor(0.1469), tensor(0.8803))

In [46]:
# Mean and standard deviation of a single input from the batch and of its features
x[0, :].mean(), x[0, :].std()

(tensor(-3.5763e-09), tensor(1.0000))